In [2]:
import pandas as pd
import sqlite3

In [3]:
path = "Data/im.db"
conn = sqlite3.connect(path)
cur = conn.cursor()


In [4]:
pd.read_sql("""SELECT name
            FROM sqlite_master
            WHERE type = 'table';
""",conn)

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [5]:
bom = pd.read_csv("Data/bom.movie_gross.csv")
bom.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [6]:
bom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [7]:
bom['foreign_gross'] = pd.to_numeric(bom['foreign_gross'], errors='coerce')


In [8]:
print(bom['domestic_gross'].isna().sum())
print(bom['foreign_gross'].isna().sum())

28
1355


In [9]:
x= bom[bom['domestic_gross'].isna() & bom['foreign_gross'].isna()]
x

,title,studio,domestic_gross,foreign_gross,year


In [10]:
# Fill null values with 0
bom['domestic_gross'].fillna(0, inplace=True)
bom['foreign_gross'].fillna(0, inplace=True)

In [11]:
bom['studio'].unique()
bom['studio'].isna()


0       False
1       False
2       False
3       False
4       False
        ...  
3382    False
3383    False
3384    False
3385    False
3386    False
Name: studio, Length: 3387, dtype: bool

In [12]:
bom['studio'].value_counts().head()

IFC      166
Uni.     147
WB       140
Magn.    136
Fox      136
Name: studio, dtype: int64

In [13]:
#Fill null values with unknown
bom['studio'].fillna('Unknown', inplace=True)

In [14]:
# Movie Basics Table
pd.read_sql("""SELECT COUNT(*)
                FROM movie_basics
                WHERE genres IS NULL;
""", conn)

,COUNT(*)
0,0


In [15]:
pd.read_sql("""SELECT COUNT(*)
                FROM movie_basics
                WHERE genres IS NULL;
""", conn)

,COUNT(*)
0,0


In [16]:
# Fill null values with 90 since average is usually 90 minutes

cur.execute("""
    UPDATE movie_basics
    SET runtime_minutes = 90
    WHERE runtime_minutes IS NULL;
""")
conn.commit()


In [17]:
cur.execute("""
                UPDATE movie_basics
                SET genres = 'Unknown'
                WHERE genres IS NULL;
""")
conn.commit()

In [18]:
# Separate those with multiple genres using | do that they can be counted individually
cur.execute("""
                UPDATE movie_basics
                SET genres = REPLACE(genres, ',', '|');
""")
conn.commit()

In [19]:
pd.read_sql("""SELECT genres, COUNT(*) AS genre_count
                FROM movie_basics
                GROUP BY genres
                ORDER BY genre_count DESC;
""", conn)

,genres,genre_count
0,Documentary,32185
1,Drama,21486
2,Comedy,9177
3,Unknown,5408
4,Horror,4372
...,...,...
1081,Action|Animation|Music,1
1082,Action|Animation|History,1
1083,Action|Animation|Documentary,1
1084,Action|Animation|Biography,1


In [20]:
# To count all movies with the genre Documentary
pd.read_sql("""SELECT COUNT(*) as documentaries
                FROM movie_basics
                WHERE genres LIKE '%Documentary%';
""", conn)

,documentaries
0,51640


In [21]:
pd.read_sql("""SELECT COUNT(*)
                FROM movie_ratings
                WHERE movie_id IS NULL
                OR averagerating IS NULL
                OR numvotes IS NULL;
""", conn)

,COUNT(*)
0,0


In [22]:
pd.read_sql("""SELECT COUNT(*)
                FROM movie_akas
                WHERE movie_id IS NULL
                OR ordering IS NULL
                OR title IS NULL;
""", conn)

,COUNT(*)
0,0


In [23]:
pd.read_sql("""SELECT COUNT(*)
                FROM directors
                WHERE movie_id IS NULL
                OR person_id IS NULL;
""", conn)

,COUNT(*)
0,0


In [24]:
tmdb = pd.read_csv("Data/tmdb.movies.csv")
tmdb.head()
tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


In [25]:
## SQL table relations
#Length of movies
pd.read_sql("""SELECT *
                FROM movie_basics
                ORDER BY runtime_minutes DESC
                LIMIT 10;
""", conn)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt8273150,Logistics,Logistics,2012,51420.0,Documentary
1,tt2659636,Modern Times Forever,Modern Times Forever,2011,14400.0,Documentary
2,tt7492094,Nari,Nari,2017,6017.0,Documentary
3,tt5068890,Hunger!,Hunger!,2015,6000.0,Documentary|Drama
4,tt5136218,London EC1,London EC1,2015,5460.0,Comedy|Drama|Mystery
5,tt5240738,Ember Glow,Ember Glow,2015,4980.0,Drama|Western
6,tt1735956,Deregulation,Foreclose,2012,4200.0,Comedy|Drama
7,tt6150204,Nieuwe Tieten,Nieuwe Tieten,2013,4080.0,Documentary|Drama
8,tt1674154,City of Eternal Spring,City of Eternal Spring,2010,3450.0,Documentary|Drama|News
9,tt3837350,A 2nd generation film,A 2nd generation film,2013,3077.0,Drama


In [26]:
# Average ratings by genre
pd.read_sql("""SELECT
            genres, AVG(averagerating) AS avg_rating, SUM(numvotes) AS total_votes, COUNT(original_title) AS num_movies
                FROM movie_basics
                JOIN movie_ratings 
            ON movie_basics.movie_id= movie_ratings.movie_id
            GROUP BY genres
            ORDER BY avg_rating DESC
            LIMIT 10;
""", conn)

,genres,avg_rating,total_votes,num_movies
0,Comedy|Documentary|Fantasy,9.4,5,1
1,Documentary|Family|Musical,9.3,19,1
2,History|Sport,9.2,5,1
3,Music|Mystery,9.0,5,1
4,Game-Show,9.0,7,1
5,Drama|Short,8.8,8,1
6,Drama|Fantasy|War,8.8,22,1
7,Documentary|News|Sport,8.8,25,1
8,Documentary|News|Reality-TV,8.8,8,1
9,Comedy|Drama|Reality-TV,8.8,15,1


In [27]:
# # Read from database
# df = pd.read_sql("""
#     SELECT original_title, genres, averagerating
#     FROM movie_basics
#     JOIN movie_ratings USING (movie_id)
# """, conn)

# # Split on '/' and expand rows
# df = df.assign(genre=df['genres'].str.split('/')).explode('genre')

# # Now group by the individual genre
# result = (
#     df.groupby('genre', as_index=False)
#       .agg(avg_rating=('averagerating', 'mean'),
#            num_movies=('original_title', 'count'))
#       .sort_values('avg_rating', ascending=False)
# )
# print(result)


In [28]:
pd.read_sql("""SELECT
            genres, ROUND(AVG(averagerating),2) AS avg_rating, SUM(numvotes) AS total_votes, COUNT(original_title) AS num_movies
            FROM movie_basics
            JOIN movie_ratings 
                ON movie_basics.movie_id= movie_ratings.movie_id
            WHERE numvotes > 1000
            GROUP BY genres
            HAVING COUNT(original_title) > 2
            ORDER BY avg_rating DESC
            LIMIT 10;
""", conn)

,genres,avg_rating,total_votes,num_movies
0,Musical,8.18,8620,5
1,Documentary|History|Sport,8.03,5668,3
2,Documentary|Family,8.00,7987,3
3,Documentary|News,7.93,39758,6
4,Adventure|Biography|Documentary,7.93,19737,6
5,Documentary|History|War,7.90,29488,5
6,Documentary|Drama|News,7.87,8928,3
7,Comedy|Documentary|History,7.83,4384,3
8,Documentary|Sport,7.71,86138,25
9,Animation|Drama|Fantasy,7.68,226695,8


In [29]:
# Number of votes
pd.read_sql("""SELECT
            genres, ROUND(AVG(averagerating),2) AS avg_rating, SUM(numvotes) AS total_votes, COUNT(original_title) AS num_movies
            FROM movie_basics
            JOIN movie_ratings 
                ON movie_basics.movie_id= movie_ratings.movie_id
            WHERE numvotes > 1000
            GROUP BY genres
            HAVING COUNT(original_title) > 2
            ORDER BY total_votes DESC
            LIMIT 10;
""", conn)

,genres,avg_rating,total_votes,num_movies
0,Action|Adventure|Sci-Fi,6.18,23015643,77
1,Action|Adventure|Fantasy,5.69,9649573,64
2,Adventure|Animation|Comedy,6.36,8656839,122
3,Comedy|Drama|Romance,6.35,7514961,331
4,Action|Adventure|Comedy,5.63,7225420,90
5,Drama,6.76,7165635,752
6,Comedy,5.66,6120493,549
7,Comedy|Drama,6.54,6092942,471
8,Action|Crime|Drama,6.28,5503533,191
9,Drama|Romance,6.57,5338598,326


In [30]:
# Popularity Index(combines votes and rate into weighted popularity score)
# popularity score = average rating * numvotes / (numvotes + k) -> standardizing vote counts
pd.read_sql("""SELECT
                genres, AVG(averagerating * (numvotes *1.0/(numvotes+1000))) AS popularity_score, SUM(numvotes) AS total_votes
            FROM movie_basics
            JOIN movie_ratings 
                ON movie_basics.movie_id= movie_ratings.movie_id
            GROUP BY genres
            ORDER BY popularity_score DESC
            ;
""", conn)

,genres,popularity_score,total_votes
0,Documentary|Sport|Thriller,7.636482,28979
1,Drama|War|Western,6.149059,5851
2,Biography|Music|Romance,5.858862,3570
3,Comedy|Music|Thriller,5.852482,5587
4,Biography|Family|Sport,5.577648,3438
...,...,...,...
919,Family|War,0.028856,5
920,Horror|Musical|Romance,0.025871,5
921,Action|Crime|Musical,0.021891,5
922,Adventure|Crime|Romance,0.020515,9


In [31]:
pd.read_sql("""SELECT
            start_year, genres, ROUND(AVG(averagerating),2) AS avg_rating, SUM(numvotes) AS total_votes
            FROM movie_basics
            JOIN movie_ratings 
                ON movie_basics.movie_id= movie_ratings.movie_id
            GROUP BY start_year, genres
            ORDER BY avg_rating DESC;
""",conn)

,start_year,genres,avg_rating,total_votes
0,2019,Biography|Documentary|Drama,9.8,28
1,2010,Documentary|Family|Romance,9.7,25
2,2019,Comedy|Documentary|Sport,9.7,22
3,2019,Documentary|Drama|Sport,9.6,12
4,2015,Comedy|Documentary|Fantasy,9.4,5
...,...,...,...,...
4616,2012,Action|Biography|History,1.3,33
4617,2011,Action|Fantasy|Sci-Fi,1.2,303
4618,2014,Comedy|History|Horror,1.1,168
4619,2011,Adventure|Biography|Comedy,1.0,352


In [32]:
pd.read_sql("""SELECT 
        CASE
           WHEN runtime_minutes < 90 THEN 'Short (<90)'
           WHEN runtime_minutes BETWEEN 90 AND 120 THEN 'Medium (90-120)'
           ELSE 'Long (>120)'
        END AS runtime_category,
        AVG(averagerating) AS avg_rating
        FROM movie_basics
        JOIN movie_ratings 
            USING (movie_id)
        GROUP BY runtime_category
        ORDER BY avg_rating DESC;
""", conn)

,runtime_category,avg_rating
0,Long (>120),6.477179
1,Short (<90),6.393324
2,Medium (90-120),6.264974


In [37]:
pd.read_sql("""SELECT primary_name AS director,
                    AVG(averagerating) AS avg_rating,
                    COUNT(movie_id) AS movies_directed
                FROM directors 
                JOIN persons 
                    USING (person_id)
                JOIN movie_ratings 
                    USING (movie_id)
                GROUP BY primary_name
                HAVING movies_directed > 3
                ORDER BY avg_rating DESC
                LIMIT 10;
""", conn)

,director,avg_rating,movies_directed
0,D. Todd Deeken,9.500000,4
1,David Beriain,9.300000,4
2,Fan Li,9.214286,7
3,Damir Radic,9.200000,7
4,Zach Logan,9.200000,4
5,Nathan D. Myers,9.200000,4
6,Marjolein Hermans,9.200000,4
7,Kris Flink,9.200000,5
8,Kara Kwiecinski,9.200000,5
9,Greg Werner,9.200000,4


In [ ]:
# pd.read_sql("""SELECT
#     original_title,
#     genres,
#     averagerating,
#     numvotes,
#     ((numvotes * averagerating + 1000 * 6.5) / (numvotes + 1000)) AS bayesian_score
# FROM movie_basics
# JOIN movie_ratings
#     USING (movie_id)
# ORDER BY bayesian_score DESC
# LIMIT 10;
# """, conn)

,original_title,genres,averagerating,numvotes,bayesian_score
0,Dag II,Action|Drama|War,9.3,100568,9.272432
1,Once Upon a Time ... in Hollywood,Comedy|Drama,9.7,5600,9.215152
2,Aynabaji,Crime|Mystery|Thriller,9.3,18470,9.156189
3,Wheels,Drama,9.3,17308,9.147061
4,CM101MMXI Fundamentals,Comedy|Documentary,9.2,41560,9.136560
5,Peranbu,Drama,9.4,9629,9.127162
6,Aloko Udapadi,Drama|History,9.5,6509,9.100479
7,Inception,Action|Adventure|Sci-Fi,8.8,1841066,8.798751
8,Avengers: Endgame,Action|Adventure|Sci-Fi,8.8,441135,8.794798
9,O.J.: Made in America,Biography|Crime|Documentary,8.9,14946,8.749492


In [ ]:
'''The results show, genres with the highest average rating and genres with the highest number of votes
    The results then grouped the runtimes with the hightest average rating
    The results grouped directors with average rating

'''